<b>In this notebook calculations for 5-grams and 8-grams should be done separately not to overload the memory and kill the kernel!</b>

In [1]:
import os
from timeit import default_timer as timer

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from Bio import SeqIO
from gensim.models import Word2Vec

import multiprocessing

### Read .fasta files (49000 genes)

In [2]:
path_gene_nuc = '../../sf_data/prodigal_output/gene_nuc/'
organisms_names = [f[:f.index('_', f.index('_') + 1)] for f in os.listdir(path_gene_nuc)]
full_path_gene_nuc = [path_gene_nuc + f for f in os.listdir(path_gene_nuc)]

In [3]:
organisms = [SeqIO.parse(gene_nuc_file, 'fasta') for gene_nuc_file in full_path_gene_nuc]
org_gene_lst = []

for (id_org, organism), org_name in zip(enumerate(organisms), organisms_names):
    for gene_record in organism:
        org_gene_lst.append(('_'.join((str(id_org), org_name)), gene_record.id,
                             str(gene_record.seq).replace('N', '')))

In [4]:
del path_gene_nuc
del organisms
del organisms_names
del full_path_gene_nuc

In [5]:
len(org_gene_lst)

49807

In [6]:
org_gene_lst[7]

('0_Bacillus_amyloliquefaciens',
 'NC_014551.1_8',
 'ATGACGTCAAATCATCATGCCCCTTATGACCTGGGCTACACACGTGCTACAATGGGCAGAACAAAGGGCAGCGAAACCGCGAGGTTAAGCCAATCCCACAAATCTGTTCTCAGTTCGGATCGCAGTCTGCAACTCGACTGCGTGAAGCTGGAATCGCTAGTAATCGCGGATCAGCATGCCGCGGTGAATACGTTCCCGGGCCTTGTACACACCGCCCGTCACACCACGAGAGTTTGTAACACCCGAAGTCGGTGA')

### Extract ngrams (5grams and 8grams)

In [7]:
def ngram_extractor2(org_g_lst, n_min=3, n_max=3):
    ngram_range = list(range(min(n_min,n_max), max(n_min+1, n_max+1)))
    
    ngrams_result = []
    
    start = timer()
    for (g_num, (org, g_id, g_seq)) in enumerate(org_g_lst):
        if(g_num) % 100 == 0:
            print(f'time: {timer()-start:.2} sec\tid_gene={g_num}')
        
        ngrams_with_order = []
        
        [ngrams_with_order.append(g_seq[indx:indx+n])
         for indx in range(len(g_seq) - (n_min-1))
         for n in ngram_range
         if (indx+n) <= len(g_seq)]
        
        ngrams_result.append(ngrams_with_order)
        
    return ngrams_result

In [40]:
all_genes_5grams = ngram_extractor2(org_gene_lst, n_min=5, n_max=5)

In [9]:
len(all_genes_5grams)

49807

In [ ]:
# del all_genes_5grams

In [ ]:
all_genes_8grams = ngram_extractor2(org_gene_lst, n_min=8, n_max=8)

In [ ]:
len(all_genes_8grams)

In [ ]:
# del all_genes_8grams

### Train word2vec model (on 49000 genes, 5grams or 8grams)

#### 5-grams

In [10]:
path_w2v_model_5 = 'dna2vec_data/word2vec_49000genes_5grams_size300.model'

In [11]:
num_features = 300
context_size = 5
min_count = 1
num_workers = multiprocessing.cpu_count()
seed = 42

In [12]:
model_5 = Word2Vec(size=num_features,
                 window=context_size,
                 min_count=min_count,
                 workers=num_workers,
                 seed=seed)

model_5.save(path_w2v_model_5)

In [13]:
model_5.build_vocab(all_genes_5grams)
model_5.save(path_w2v_model_5)

In [14]:
model_5.corpus_count, model_5.epochs

(49807, 5)

In [15]:
#about 16 min
start = timer()
model_5.train(all_genes_5grams, total_examples=model_5.corpus_count, epochs=model_5.epochs)
model_5.save(path_w2v_model_5)
print(f'total time: {timer()-start:.2} sec')

total time: 9.7e+02 sec


In [ ]:
# if loading pretrained model:
# model_5 = Word2Vec.load(path_w2v_model_5)

In [ ]:
# del model_5

#### 8-grams

In [ ]:
path_w2v_model_8 = 'dna2vec_data/word2vec_49000genes_8grams_size300.model'

In [12]:
model_8 = Word2Vec(size=num_features,
                 window=context_size,
                 min_count=min_count,
                 workers=num_workers,
                 seed=seed)

model_8.save(path_w2v_model_8)

In [13]:
model_8.build_vocab(all_genes_8grams)
model_8.save(path_w2v_model_8)

In [14]:
model_8.corpus_count, model_8.epochs

(49807, 5)

In [ ]:
#about 26 min
start = timer()
model_8.train(all_genes_8grams, total_examples=model_8.corpus_count, epochs=model_8.epochs)
model_8.save(path_w2v_model_8)
print(f'total time: {timer()-start:.2} sec')

In [ ]:
# if loading pretrained model:
# model_8 = Word2Vec.load(path_w2v_model_8)

### Read .csv file (1000 genes)

In [16]:
df_genes = pd.read_csv('~/sf_data/all_genes.csv', header=None)
df_genes.shape

(1066, 4)

In [18]:
df_genes = df_genes.iloc[:, :3]
df_genes.head()

,0,1,2
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...


In [19]:
df_genes[0].unique()

array(['Bacillus_amyloliquefaciens_GCF_000196735.1_ASM19673v1_genomic',
       'Bacillus_atrophaeus_GCF_000742675.1_ASM74267v1_genomic',
       'Bacillus_halotolerans_GCF_001517105.1_ASM151710v1_genomic',
       'Bacillus_licheniformis_GCF_000011645.1_ASM1164v1_genomic',
       'Bacillus_mojavensis_GCF_000245335.1_ASM24533v1_genomic',
       'Bacillus_paralicheniformis_GCF_000408885.1_ASM40888v1_genomic',
       'Bacillus_siamensis_GCF_000262045.1_KCTC_13613_01_genomic',
       'Bacillus_sonorensis_GCF_002202015.1_ASM220201v1_genomic',
       'Bacillus_subtilis_GCF_000009045.1_ASM904v1_genomic',
       'Bacillus_tequilensis_GCF_000507145.1_KCTC_13622_01_genomic',
       'Bacillus_vallismortis_GCF_000245315.1_ASM24531v1_genomic',
       'Bacillus_velezensis_GCF_002117165.1_ASM211716v1_genomic'],
      dtype=object)

In [20]:
orgs = ['_'.join(org_name.split('_',2)[0:2]) for org_name in df_genes[0]]
len(orgs)

1066

In [21]:
path_gene_nuc = '../../sf_data/prodigal_output/gene_nuc/'
org_names = [f[:f.index('_', f.index('_') + 1)] for f in os.listdir(path_gene_nuc)]
org_names

['Bacillus_amyloliquefaciens',
 'Bacillus_atrophaeus',
 'Bacillus_halotolerans',
 'Bacillus_licheniformis',
 'Bacillus_mojavensis',
 'Bacillus_paralicheniformis',
 'Bacillus_siamensis',
 'Bacillus_sonorensis',
 'Bacillus_subtilis',
 'Bacillus_tequilensis',
 'Bacillus_vallismortis',
 'Bacillus_velezensis']

In [22]:
# check sets of organisms names in two files
print(set(orgs) ^ set(org_names))
print(set(org_names) ^ set(orgs))

set()
set()


In [23]:
tmp_org = []

for name in df_genes[0]:
    for (org_id, org_name) in enumerate(org_names):
        if name.startswith(org_name):
            tmp_org.append('_'.join((str(org_id), org_name)))

In [24]:
df_genes['org_id_name'] = tmp_org
df_genes.head()

,0,1,2,org_id_name
0,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,Bacillus_amyloliquefaciens_GCF_000196735.1_ASM...,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


In [26]:
del tmp_org

In [27]:
df_genes = df_genes.iloc[:, 1:]
df_genes.head()

,1,2,org_id_name
0,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


In [28]:
df_genes.rename(columns={1:'gene_name',2:'gene_seq',3:'org_id_name'}, inplace=True)
df_genes.head()

,gene_name,gene_seq,org_id_name
0,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


In [29]:
df_meta_info = df_genes.loc[:, ['org_id_name', 'gene_name']]
df_meta_info.head()

,org_id_name,gene_name
0,0_Bacillus_amyloliquefaciens,pgk
1,0_Bacillus_amyloliquefaciens,rplW
2,0_Bacillus_amyloliquefaciens,rplE
3,0_Bacillus_amyloliquefaciens,rplC
4,0_Bacillus_amyloliquefaciens,rplF


In [30]:
df_meta_info.to_csv('dna2vec_data/meta_info_1066_genes.tsv', sep='\t', index=False)

In [31]:
df_genes.head()

,gene_name,gene_seq,org_id_name
0,pgk,ATGAATAAGAAAACAGTAAAAGACATCGACGTAAAAGGCAAAGTCG...,0_Bacillus_amyloliquefaciens
1,rplW,ATGAAAGATCCTCGTGATGTTCTTAAGCGCCCCGTCATTACTGAAC...,0_Bacillus_amyloliquefaciens
2,rplE,ATGAACCGCCTTAAAGAAAAGTACAATAAAGAAATTTCACCTGCTT...,0_Bacillus_amyloliquefaciens
3,rplC,ATGACCAAAGGAATCTTAGGAAGAAAAATTGGTATGACGCAAGTAT...,0_Bacillus_amyloliquefaciens
4,rplF,ATGTCTCGTGTAGGTAAGAAACTGCTTGAGATCCCTTCTGAAGTTA...,0_Bacillus_amyloliquefaciens


### extract 5grams (or 8grams) from 1000 genes

In [32]:
def ngram_extractor(g_seq_lst, n_min=3, n_max=3):
    ngram_range = list(range(min(n_min,n_max), max(n_min+1, n_max+1)))
    
    ngrams_result = []
    
    start = timer()
    for (g_num, g_seq) in enumerate(g_seq_lst):
        if(g_num) % 100 == 0:
            print(f'time: {timer()-start:.2} sec\tid_gene={g_num}')
        
        ngrams_with_order = []
        
        [ngrams_with_order.append(g_seq[indx:indx+n])
         for indx in range(len(g_seq) - (n_min-1))
         for n in ngram_range
         if (indx+n) <= len(g_seq)]
        
        ngrams_result.append(ngrams_with_order)
        
    return ngrams_result

In [33]:
genes_seq = [gene_seq for gene_seq in df_genes['gene_seq']]
genes_seq[7]

'ATGGTTATGACAGATCCAATTGCAGATATGCTGACTCGTATTCGTAATGCAAACATGGTACGTCATGAGAAGCTTGAAATTCCTGCTTCTAAATTGAAAAGAGAAATTGCTGACATTTTAAAGCGTGAAGGTTTCATTCGTGACGTTGAGTTCGTAGAAGACAGCAAACAAGGTATCATCCGCGTTTTCTTGAAATACGGACAAAACAACGAGCGCGTTATCACTGGTCTTAAAAGAATCAGCAAACCAGGTTTGCGTGTATACGCTAAATCAAATGAAGTACCTCGCGTACTTAACGGTCTTGGAATCGCGATTATTTCTACATCACAAGGTGTTTTAACGGACAAAGAAGCCCGTGCAAAACAAGCTGGTGGAGAAGTTCTAGCATACGTTTGGTAA'

In [34]:
genes_5grams = ngram_extractor(genes_seq, n_min=5, n_max=5)

time: 2.1e-06 sec	id_gene=0
time: 0.16 sec	id_gene=100
time: 0.26 sec	id_gene=200
time: 0.35 sec	id_gene=300
time: 0.45 sec	id_gene=400
time: 0.56 sec	id_gene=500
time: 0.65 sec	id_gene=600
time: 0.74 sec	id_gene=700
time: 0.87 sec	id_gene=800
time: 0.97 sec	id_gene=900
time: 1.1 sec	id_gene=1000


In [ ]:
genes_8grams = ngram_extractor(genes_seq, n_min=8, n_max=8)

### gene2vec (for 1000 genes)

#### 5-grams

In [35]:
genes_5grams_vec = []
for gene in genes_5grams:
    genes_5grams_vec.append(np.vstack([model_5.wv[ngram] for ngram in gene]).mean(axis=0))

In [36]:
len(genes_5grams_vec)

1066

In [37]:
len(genes_5grams_vec[0])

300

In [39]:
pd.DataFrame(genes_5grams_vec).to_csv('dna2vec_data/genes_vec_5grams_1066_genes.tsv', sep='\t', header=False, index=False)

#### 8-grams

In [ ]:
genes_8grams_vec = []
for gene in genes_8grams:
    genes_8grams_vec.append(np.vstack([model_8.wv[ngram] for ngram in gene]).mean(axis=0))

In [36]:
len(genes_8grams_vec)

1066

In [37]:
len(genes_8grams_vec[0])

300

In [39]:
pd.DataFrame(genes_8grams_vec).to_csv('dna2vec_data/genes_vec_8grams_1066_genes.tsv', sep='\t', header=False, index=False)